In [ ]:
####*****************************************etape 1 Observation des resultats de SPERT et des vrais resultats par phrase
import json
import numpy as np
import pandas as pd
import glob
entities_types =["No","Loc","Org", "Peop","Other"]
relations_types =["Work_For", "Kill", "OrgBased_In", "Live_In", "Located_In"]



print(type(json))
with open("CONLL04\conll04_test.json") as file :
    data_reals = json.load(file)
with open("CONLL04/logs/spert_treshold_0.4/predictions_valid_epoch_20.json") as file :
    data_preds = json.load(file)
with open("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_predictions_valid_epoch_20.json") as file :
    data_neur = json.load(file)  
    
def study (data_reals,data_neur):
    longn1 =0
    longn2 =0
    longn3 =0
    longr3 =0
    longr2 =0
    longr1 =0
    longnr3 =0
    longnr2 =0
    longnr1 =0
    longr =0
    longn=0
    longnr=0
    for doc_id , doc_real in enumerate(data_reals):
        entities_pred ={}
        entities_real ={}    
        for entity in doc_real["entities"]:
            entity_pos= (entity["start"],entity["end"])
            entity_words = (' '.join(doc_real["tokens"][entity["start"]:entity["end"]]))
            entities_real[str(entity_pos)] = (entity_words, entity["type"].lower())
        for entity in data_preds[doc_id]["entities"]:
            entity_pos= (entity["start"],entity["end"])
            entity_words = (' '.join(doc_real["tokens"][entity["start"]:entity["end"]]))
            entity_probs = entity["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(entity_probs, dtype=float, sep=' ')
            prob =probs[entities_types.index(entity['type'])]
            entities_pred[str(entity_pos)] =(prob , (entity_words, entity["type"].lower()) )

        for idx in entities_real:
            if idx not in entities_pred:
                entities_pred[idx]=(0,"0***")
        for idx in entities_pred:
            if idx not in entities_real:
                entities_real[idx]=(0,"0***")
        display_real=[]
        display_pred =[]
        positions=[]
        for idx in entities_real :
            display_real.append(entities_real[idx])
            display_pred.append(entities_pred[idx])
            positions.append(idx)
            longn+=1
            if entities_real[idx]!=entities_pred[idx][1] and entities_real[idx][1]!="0***"  and entities_pred[idx][1]!="0***" :
                longn3+=1
            if entities_real[idx]!=entities_pred[idx][1] and  entities_pred[idx][1]=="0***" :
                longn2+=1  # il n'identifie pas l'entité
            if entities_real[idx]!=entities_pred[idx][1] and  entities_real[idx][1]=="0***" :
                longn1+=1  # il identifie l'entité 
        df ={"position":positions , "real":display_real , "predictions":display_pred}
        df =pd.DataFrame(data=df)
        print("doc n°:",doc_id)
        print(df)

        
        relations_pred ={}
        relations_real ={}    
        for relation in doc_real["relations"]:
            relation_pos= (doc_real["entities"][relation["head"]]["start"], doc_real["entities"][relation["head"]]["end"]
                           ,doc_real["entities"][relation["tail"]]["start"],doc_real["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][doc_real["entities"][relation["head"]]["start"]:doc_real["entities"][relation["head"]]["end"]]),
                           ' '.join(doc_real["tokens"][doc_real["entities"][relation["tail"]]["start"]:doc_real["entities"][relation["tail"]]["end"]]))
            relations_real[str(relation_pos)] = (relation_words, relation["type"].lower())
        for relation in data_preds[doc_id]["relations"]:
            relation_pos= (data_preds[doc_id]["entities"][relation["head"]]["start"], data_preds[doc_id]["entities"][relation["head"]]["end"]
                           ,data_preds[doc_id]["entities"][relation["tail"]]["start"],data_preds[doc_id]["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["head"]]["start"]:data_preds[doc_id]["entities"][relation["head"]]["end"]]),
                   ' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["tail"]]["start"]:data_preds[doc_id]["entities"][relation["tail"]]["end"]]))
            relation_probs = relation["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(relation_probs, dtype=float, sep=' ')
            prob =probs[relations_types.index(relation['type'])]
            relations_pred[str(relation_pos)] = (prob, relation_words, relation["type"].lower())

        for idx in relations_real:
            if idx not in relations_pred:
                relations_pred[idx]=(0,"0***")
        for idx in relations_pred:
            if idx not in relations_real:
                relations_real[idx]=(0,"0***")
        display_real=[]
        display_pred =[]
        positions=[]
        for idx in relations_real :
            display_real.append(relations_real[idx])
            display_pred.append(relations_pred[idx])
            positions.append(idx)
            longr+=1
            if relations_real[idx]!=relations_pred[idx][1:] and relations_real[idx][1]!="0***"  and relations_pred[idx][1]!="0***" :
                longr3+=1
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_pred[idx][1]=="0***" :
                longr2+=1  # il n'identifie pas l'entité
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_real[idx][1]=="0***" :
                longr1+=1  # il identifie l'entité 
        
        df ={"position":positions , "real":display_real , "predictions":display_pred}
        df =pd.DataFrame(data=df)
        print("doc n°:",doc_id)
        print(df)
        
        
        
        relations_pred ={}
        relations_real ={}    
        for relation in doc_real["relations"]:
            relation_pos= (doc_real["entities"][relation["head"]]["start"], doc_real["entities"][relation["head"]]["end"]
                           ,doc_real["entities"][relation["tail"]]["start"],doc_real["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][doc_real["entities"][relation["head"]]["start"]:doc_real["entities"][relation["head"]]["end"]]),
                           ' '.join(doc_real["tokens"][doc_real["entities"][relation["tail"]]["start"]:doc_real["entities"][relation["tail"]]["end"]]))
            type_entity1= doc_real["entities"][relation["head"]]["type"]
            type_entity2=doc_real["entities"][relation["tail"]]["type"]
            relations_real[str(relation_pos)] = (relation_words, relation["type"].lower(), type_entity1.lower(),type_entity2.lower())
        
        for relation in data_preds[doc_id]["relations"]:
            relation_pos= (data_preds[doc_id]["entities"][relation["head"]]["start"], data_preds[doc_id]["entities"][relation["head"]]["end"]
                           ,data_preds[doc_id]["entities"][relation["tail"]]["start"],data_preds[doc_id]["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["head"]]["start"]:data_preds[doc_id]["entities"][relation["head"]]["end"]]),
                   ' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["tail"]]["start"]:data_preds[doc_id]["entities"][relation["tail"]]["end"]]))
            relation_probs = relation["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(relation_probs, dtype=float, sep=' ')
            prob =probs[relations_types.index(relation['type'])]
            
            entity1 = data_preds[doc_id]["entities"][relation["head"]]
            entity2 = data_preds[doc_id]["entities"][relation["tail"]]
            entity_probs = entity1["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(entity_probs, dtype=float, sep=' ')
            probe1 =probs[entities_types.index(entity1['type'])]
            
            entity_probs = entity2["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(entity_probs, dtype=float, sep=' ')
            probe2 =probs[entities_types.index(entity2['type'])]
            prob = prob*probe1*probe2
            
            type_entity1= entity1["type"]
            type_entity2=entity2["type"]
            relations_pred[str(relation_pos)] = (prob, relation_words, relation["type"].lower(), type_entity1.lower(),type_entity2.lower())

        for idx in relations_real:
            if idx not in relations_pred:
                relations_pred[idx]=(0,"0***")
        for idx in relations_pred:
            if idx not in relations_real:
                relations_real[idx]=(0,"0***")
        display_real=[]
        display_pred =[]
        positions=[]
        for idx in relations_real :
            display_real.append(relations_real[idx])
            display_pred.append(relations_pred[idx])
            positions.append(idx)
            longnr+=1
            if relations_real[idx]!=relations_pred[idx][1:] and relations_real[idx][1]!="0***"  and relations_pred[idx][1]!="0***" :
                longnr3+=1
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_pred[idx][1]=="0***" :
                longnr2+=1  # il n'identifie pas l'entité
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_real[idx][1]=="0***" :
                longnr1+=1  # il identifie l'entité 
                
            
            
            
            

            """if relations_pred[idx] !=  relations_real[idx] and  relations_real[idx]=="0***"  :
                display_real.append(relations_real[idx])
                display_pred.append(relations_pred[idx])
                positions.append(idx)"""



        """df ={"position":positions , "real":display_real , "predictions":display_pred}
        df =pd.DataFrame(data=df)"""
        print("doc n°:",doc_id)
        print(df)
        print("(", longn3,"**",longn2,"**",longn1,"**",longn,"**) (",longr3,"**",longr2,"**",longr1,"**",longr,"**) (",longnr3,"**",longnr2,"**",longnr1,"**",longnr,")" "****************************************************separation***********************")
        
#study(data_reals,data_preds)
study(data_reals,data_preds)

"""
print("**************************************Separtation")
thresholds=[0.4]
#print("logs\spert_treshold_"+str(threshold)+"/*.json")
for threshold in thresholds:
    files =glob.glob("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_*.json")
    for file_ in files :
        print(file_)    
        with open(file_) as file :
            data_neur  = json.load(file)
        study(data_reals,data_preds)
        print (file_)
       """

In [18]:
study(data_preds,data_neur)


doc n°: 0
   position                     real                            predictions
0    (5, 7)  (Hakawati Theatre, org)  (0.95854974, (Hakawati Theatre, org))
1  (10, 11)         (Jerusalem, loc)        (0.997233212, (Jerusalem, loc))
2  (17, 18)    (Palestinians, other)     (0.7260423, (Palestinians, other))
doc n°: 0
Empty DataFrame
Columns: [position, real, predictions]
Index: []
doc n°: 0
Empty DataFrame
Columns: [position, real, predictions]
Index: []
( 0 ** 0 ** 0 ** 3 **) ( 0 ** 0 ** 0 ** 0 **) ( 0 ** 0 ** 0 ** 0 )****************************************************separation***********************
doc n°: 1
  position            real                  predictions
0   (0, 1)  (PERUGIA, loc)  (0.9981389, (PERUGIA, loc))
1   (2, 3)    (Italy, loc)   (0.99702686, (Italy, loc))
2   (4, 5)       (AP, org)      (0.99767858, (AP, org))
doc n°: 1
       position                            real                                predictions
0  (0, 1, 2, 3)  ((PERUGIA, Italy), located_in)  

( 0 ** 0 ** 0 ** 40 **) ( 0 ** 0 ** 0 ** 7 **) ( 0 ** 0 ** 0 ** 7 )****************************************************separation***********************
doc n°: 9
   position              real                     predictions
0    (5, 6)  (Alekseev, peop)  (0.99908388, (Alekseev, peop))
1  (12, 13)     (Soviet, loc)     (0.99794573, (Soviet, loc))
2  (22, 23)     (Moscow, loc)     (0.99818867, (Moscow, loc))
doc n°: 9
           position                            real                                 predictions
0    (5, 6, 12, 13)   ((Alekseev, Soviet), live_in)    (0.7838397, (Alekseev, Soviet), live_in)
1  (22, 23, 12, 13)  ((Moscow, Soviet), located_in)  (0.43114984, (Moscow, Soviet), located_in)
doc n°: 9
           position                            real                                 predictions
0    (5, 6, 12, 13)   ((Alekseev, Soviet), live_in)    (0.7838397, (Alekseev, Soviet), live_in)
1  (22, 23, 12, 13)  ((Moscow, Soviet), located_in)  (0.43114984, (Moscow, Soviet), locat

2  (28, 30, 26, 27)         ((June Anderson, U.S.), live_in)       (0.58307016, (June Anderson, U.S.), live_in)
( 0 ** 0 ** 0 ** 66 **) ( 0 ** 0 ** 0 ** 22 **) ( 0 ** 0 ** 0 ** 22 )****************************************************separation***********************
doc n°: 17
   position                            real                                   predictions
0    (0, 2)        (Boston University, org)        (0.99458212, (Boston University, org))
1    (3, 6)  (Michael D. Papagiannis, peop)  (0.98921949, (Michael D. Papagiannis, peop))
2  (13, 15)            (100 million, other)            (0.92522502, (100 million, other))
3  (13, 16)      (100 million years, other)      (0.52120191, (100 million years, other))
4  (24, 25)                    (Earth, loc)                     (0.9484849, (Earth, loc))
doc n°: 17
       position                                               real                                        predictions
0  (3, 6, 0, 2)  ((Michael D. Papagiannis, Boston Uni

( 0 ** 0 ** 0 ** 91 **) ( 0 ** 0 ** 0 ** 36 **) ( 0 ** 0 ** 0 ** 36 )****************************************************separation***********************
doc n°: 24
   position               real                      predictions
0    (5, 6)  (California, loc)  (0.99843794, (California, loc))
1  (12, 14)    (69 mph, other)    (0.98291439, (69 mph, other))
2  (15, 16)     (Ontario, loc)     (0.99915528, (Ontario, loc))
3  (17, 19)    (57 mph, other)    (0.98426855, (57 mph, other))
4  (20, 22)     (El Toro, loc)     (0.99524802, (El Toro, loc))
doc n°: 24
Empty DataFrame
Columns: [position, real, predictions]
Index: []
doc n°: 24
Empty DataFrame
Columns: [position, real, predictions]
Index: []
( 0 ** 0 ** 0 ** 96 **) ( 0 ** 0 ** 0 ** 36 **) ( 0 ** 0 ** 0 ** 36 )****************************************************separation***********************
doc n°: 25
   position                 real                        predictions
0    (5, 7)      (2 a.m., other)      (0.87334991, (2 a.m., othe

0  (5, 6, 16, 17)  ((Onondaga, Idaho), located_in)  (0.40657726, (Onondaga, Idaho), located_in)
( 0 ** 0 ** 0 ** 122 **) ( 0 ** 0 ** 0 ** 49 **) ( 0 ** 0 ** 0 ** 49 )****************************************************separation***********************
doc n°: 32
   position                                               real                                        predictions
0    (0, 3)                          (Albert O. Harjula, peop)            (0.98843926, (Albert O. Harjula, peop))
1    (7, 8)                                   (Thomaston, loc)                     (0.99951959, (Thomaston, loc))
2  (14, 16)                              (Paul Fournier, peop)                (0.99524248, (Paul Fournier, peop))
3  (22, 28)  (Department of Inland Fisheries and Wildlife, ...  (0.9712534, (Department of Inland Fisheries an...
4  (22, 29)  (Department of Inland Fisheries and Wildlife ....  (0.711757302, (Department of Inland Fisheries ...
doc n°: 32
           position  ...                  

...

4  (16, 18)  (South Carolina, loc)  (0.99338984, (South Carolina, loc))
doc n°: 255
         position                                        real                                        predictions
0    (0, 1, 2, 3)              ((COLUMBIA, S.C.), located_in)         (0.99833858, (COLUMBIA, S.C.), located_in)
1    (4, 6, 2, 3)            ((Jesse Jackson, S.C.), live_in)       (0.70529735, (Jesse Jackson, S.C.), live_in)
2  (4, 6, 16, 18)  ((Jesse Jackson, South Carolina), live_in)  (0.98143744, (Jesse Jackson, South Carolina), ...
doc n°: 255
         position                                        real                                        predictions
0    (0, 1, 2, 3)              ((COLUMBIA, S.C.), located_in)         (0.99833858, (COLUMBIA, S.C.), located_in)
1    (4, 6, 2, 3)            ((Jesse Jackson, S.C.), live_in)       (0.70529735, (Jesse Jackson, S.C.), live_in)
2  (4, 6, 16, 18)  ((Jesse Jackson, South Carolina), live_in)  (0.98143744, (Jesse Jackson, South Carolina), ...


0  (5, 7, 10, 13)  ((Jesse Owens, Danville , Ala.), live_in)  (0.99975556, (Jesse Owens, Danville , Ala.), l...
( 0 ** 0 ** 0 ** 1030 **) ( 0 ** 0 ** 0 ** 359 **) ( 0 ** 0 ** 0 ** 359 )****************************************************separation***********************
doc n°: 263
   position                                          real                                        predictions
0    (0, 1)                           (Springsteen, peop)                  (0.99590969, (Springsteen, peop))
1    (3, 5)                             (New Jersey, loc)                    (0.99557686, (New Jersey, loc))
2  (17, 18)                                 (Bruce, peop)                         (0.8343723, (Bruce, peop))
3  (33, 38)  (Universal Declaration of Human Rights, org)  (0.48397455, (Universal Declaration of Human R...
doc n°: 263
       position                                  real                                       predictions
0  (0, 1, 3, 5)  ((Springsteen, New Jersey), live_in)  (

2  (18, 22)  (Point Pleasant , Ohio, loc)  (0.98188215, (Point Pleasant , Ohio, loc))
doc n°: 270
           position  ...                                        predictions
0  (11, 14, 18, 22)  ...  (0.99915338, (Ulysses S. Grant, Point Pleasant...

[1 rows x 3 columns]
doc n°: 270
           position  ...                                        predictions
0  (11, 14, 18, 22)  ...  (0.99915338, (Ulysses S. Grant, Point Pleasant...

[1 rows x 3 columns]
( 0 ** 0 ** 0 ** 1058 **) ( 0 ** 0 ** 0 ** 370 **) ( 0 ** 0 ** 0 ** 370 )****************************************************separation***********************
doc n°: 271
   position                                  real                                        predictions
0  (12, 14)                  (United States, loc)                 (0.98930848, (United States, loc))
1  (15, 17)                  (James Monroe, peop)                 (0.99622464, (James Monroe, peop))
2  (21, 24)  (Westmoreland County ,Virginia, loc)  (0.97455716, (Wes

0  (3, 5, 16, 19)  ((Andrew Jackson, Waxhaw , S.C.), live_in)  (0.994562209, (Andrew Jackson, Waxhaw , S.C.),...
( 0 ** 0 ** 0 ** 1084 **) ( 0 ** 0 ** 0 ** 379 **) ( 0 ** 0 ** 0 ** 379 )****************************************************separation***********************
doc n°: 279
   position                       real                               predictions
0    (7, 9)  (Benjamin Harrison, peop)  (0.990820348, (Benjamin Harrison, peop))
1  (15, 17)       (United States, loc)        (0.98139489, (United States, loc))
2  (21, 25)   (North Bend , Ohio, loc)     (0.9818188, (North Bend , Ohio, loc))
doc n°: 279
         position                                               real                                        predictions
0  (7, 9, 21, 25)  ((Benjamin Harrison, North Bend , Ohio), live_in)  (0.99371207, (Benjamin Harrison, North Bend , ...
doc n°: 279
         position                                               real                                        predictions
0  (7, 9

0  (11, 13, 14, 16)  ((Lawrence Welk, North Dakota), live_in)  (0.99978131, (Lawrence Welk, North Dakota), li...
doc n°: 286
           position                                      real                                        predictions
0  (11, 13, 14, 16)  ((Lawrence Welk, North Dakota), live_in)  (0.99978131, (Lawrence Welk, North Dakota), li...
( 0 ** 0 ** 0 ** 1111 **) ( 0 ** 0 ** 0 ** 389 **) ( 0 ** 0 ** 0 ** 389 )****************************************************separation***********************
doc n°: 287
   position                 real                      predictions
0    (1, 2)       (Indiana, loc)      (0.9965449, (Indiana, loc))
1  (11, 12)  (Indianapolis, loc)  (0.995839, (Indianapolis, loc))
doc n°: 287
         position                                   real                                        predictions
0  (11, 12, 1, 2)  ((Indianapolis, Indiana), located_in)  (0.97528821, (Indianapolis, Indiana), located_in)
doc n°: 287
         position                       

In [ ]:
with open("CONLL04\conll04_test.json") as file :
    data_reals = json.load(file)
files =glob.glob("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_predictions*.json")
for file_ in files :
    print(file_)
    with open(file_) as file :
        data_preds = json.load(file)
    print(file_)
    study(data_reals,data_preds)
"""

print(type(json))
with open("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_predictions_valid_epoch_20.json") as file :
    data_reals = json.load(file)
with open("CONLL04/logs/spert_treshold_0.4/predictions_valid_epoch_20.json") as file :
    #Scierc/logs/scierc_treshold_0.1/predictions_valid_epoch_1.json
    data_preds = json.load(file)
    
study(data_reals,data_preds)"""

In [ ]:
### enlever la classe no aussi sur les predictions 

In [ ]:

import numpy as np   
import cv2
import matplotlib.pyplot as plt
import os 
from matplotlib import *
import torchvision
import sys
sys.path.append('../../')
import time
import torch
from network import Net
from neurasp import NeurASP
import pandas as pd
from torchvision import datasets, transforms
import helper
import cv2
import glob
import json
import unicodedata

#### method 3: we don't fix anything but we can have entity moving and we add a confidence to entities since it has a higher confidence in prediction

print("***********************************************METHOD3")

m = Net()    
nnMapping = {'nerjoinre': m}  

def NeurSUP (data_JSON,file_result,sup=False, sup_Entity=False):

    if sup_Entity :
        FEdprogram = r'''
        nn(nerjoinre(1,R,IdR), [ work_For, kill, orgBased_In, live_In, located_In,no]) :- relation(R,IdR,IdE1,IdE2).
        nn(nerjoinre(1,E,IdE), [no, loc, org, peop, other]) :- entity(E,IdE,Value,Start,End).

        '''
        FEaspProgram = r'''
        entity(E,IdE,Value) :- entity(E,IdE,Value,Start,End).
        
        % we remove near span duplication
        
        nested(IdE2, IdE1) :- entity(E,IdE1,_,Start1,End1),entity(E,IdE2,_,Start2,End2), ((Start1 - Start2)*(End1-End2))=1, (Start2 - End2)>1.
        nested(IdE1, IdE2) :- entity(E,IdE1,_,Start1,End1),entity(E,IdE2,_,Start2,End2), (Start2 - Start1)*(End2-End1)=1 , (Start1-End1)>1.
        nerjoinre(0,E,IdE1,no) :- entity(E,IdE1,Value),entity(E,IdE2,Value),nested(IdE2, IdE1).
        nerjoinre(0,E,IdE2,no) :- entity(E,IdE1,Value),entity(E,IdE2,Value),nested(IdE1, IdE2).
        
        % an entity can't have two types  
         
         test(Value, Named) :- nerjoinre(0,E,IdE,Named) , entity(E,IdE,Value).
         test_1(Value) :- entity(E,IdE,Value).
         count_value(Value, S) :- S=#count{E : test(Value, E)} ,test_1(Value).
         :- count_value(Value, S), S>1 .

         
        % if a relation has an entity no, it should be assign no directly. 
        nerjoinre(0,R,IdR,no):-relation(R,IdR,IdE1,IdE2), entity(E,IdE1,_), nerjoinre(0,E,IdE1,no).
        nerjoinre(0,R,IdR,no):-relation(R,IdR,IdE1,IdE2), entity(E,IdE2,_), nerjoinre(0,E,IdE2,no).

        % the type assignment to each entity should be consistent with it's neighbour relation

        group(IdR,R1,E1,E2):- relation(R,IdR,IdE1,IdE2),nerjoinre(0,E,IdE1,E1), nerjoinre(0,E,IdE2,E2),nerjoinre(0,R,IdR,R1).
        :- relation(R,IdR,IdE1,IdE2),nerjoinre(0,E,IdE2,E2),nerjoinre(0,E,IdE1,E1), entity(e,IdE1,_), entity(e,IdE2,_), not group(IdR,no,E1,E2), 
        not group(IdR,work_For,peop,org), not group(IdR,kill,peop,peop), not group(IdR,orgBased_In,org,loc),
        not group(IdR,live_In,peop,loc), not group(IdR,located_In,loc,loc) .
        '''

    #i add relation(R,IdR,IdE1,IdE2), entity(e,IdE1,_,E1), entity(e,IdE2,_,E2), not group(IdR,no,E1,E2) from the previous test
 
    
    for doc_id , document in enumerate(data_JSON[212:213]):
        doc_id=212
        #document =data_JSON[1]
        #doc_id=28
        Factlist=""
        DataList={}
        entities=[]
        entities_with_all=document["entities"]
        for idx, entity_brut in enumerate(entities_with_all):
            entity_words = (''.join(document["tokens"][entity_brut["start"]:entity_brut["end"]])).replace("ó","o")
            text = unicodedata.normalize('NFD', entity_words)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
            entity_words = str(text)
            entity_probs = entity_brut["probs"].split("[")[1].split("]")[0]
            entity_type = str(entity_brut["type"])
            Factlist=Factlist+"entity(e,e"+str(idx)+",'b_"+(''.join(e for e in entity_words if e.isalnum())).lower()+"',"+str(entity_brut["start"])+","+str(entity_brut["end"])+").\n "
            probs =1*np.fromstring(entity_probs, dtype=float, sep=' ')
            #probs[0]=0.2
            #print("********",probs)
            DataList['e,e'+str(idx)]=torch.tensor([probs], dtype=torch.float64)

            
            #**********to be back *********DataList['e,e'+str(idx)]=torch.tensor([probs], dtype=torch.float64)

        # relation(r,r1,e1,e2)ZZ
        relations_with_all=document["relations"]
        for idx ,relation_brut in enumerate(relations_with_all):
            #print("**rel**",relation_brut)
            relation_probs = relation_brut["probs"].split("[")[1].split("]")[0]
            probs= np.fromstring(relation_probs, dtype=float, sep=' ')
            probs =np.append(probs, [max(probs)-0.2])
            DataList['r,r'+str(idx)] = torch.tensor([probs], dtype=torch.float64)
            Factlist=Factlist+"relation(r,r"+str(idx)+",e"+str(relation_brut["head"])+",e"+str(relation_brut["tail"])+").\n "
        Factlist=[Factlist]
        DataList=[DataList]
        #print("datalist***",DataList)
        #print("factlist***",Factlist)

        for idx, facts in enumerate(Factlist):
            # Initialize NeurASP object
            
            NeurASPobj = NeurASP(FEdprogram + facts, nnMapping, optimizers=None)
            # Find the most probable stable model
            models = NeurASPobj.infer(dataDic=DataList[idx], obs='', mvpp=FEaspProgram + facts)
            print("**",models)
        bool=True
        if bool :
            for model in models[0]:
                for atom in model:
                    if 'nerjoinre(0,r,r' in atom:
                        relation_id= atom.split("nerjoinre(0,r,r")[1].split(",")[0]
                        data_JSON[doc_id]["relations"][int(relation_id)]["type"]= atom.split(",")[3].split(")")[0].upper()[:1] + atom.split(",")[3].split(")")[0][1:]
                    if 'nerjoinre(0,e,e' in atom :
                        entity_id= atom.split("nerjoinre(0,e,e")[1].split(",")[0]
                        #print(doc_id,"**",entity_id, "**", data_JSON[doc_id]["entities"])
                        data_JSON[doc_id]["entities"][int(entity_id)]["type"]= atom.split(",")[3].split(")")[0][:1].upper() + atom.split(",")[3].split(")")[0][1:]
                removed=0
                if sup_Entity:
                    entity_with_all =[]
                    for idx,entity in enumerate(data_JSON[doc_id]["entities"]):
                        if entity["type"]!="No":
                            entity_with_all.append(entity)
                            removed+=1
                            for relation in data_JSON[doc_id]["relations"] :
                                if relation["head"]>= idx:
                                    relation["head"]-=1
                                if relation["tail"]>= idx:
                                    relation["tail"]-=1
                            
                    data_JSON[doc_id]["entities"]=entity_with_all
                    
                if sup :
                    relation_with_all =[]
                    for relation in data_JSON[doc_id]["relations"]:
                        if relation["type"]!="No":
                            relation_with_all.append(relation)

                    data_JSON[doc_id]["relations"]=relation_with_all

                        #data_JSON[doc_id]["relations"].remove(int(relation_id))
                

                    

    #print(data_JSON[:2])
    with open(file_result, 'w') as f:
        json.dump(data_JSON, f)
        print("*********** You can actually test your model with evaluation functions")

print("ok")

thresholds=[0.4]
#print("logs\spert_treshold_"+str(threshold)+"/*.json")
for threshold in thresholds:
    files =glob.glob("CONLL04\logs\spert_treshold_"+str(threshold)+"\predictions*.json")
    for file_ in files :
        print(file_)
        with open("CONLL04/logs/spert_treshold_0.4/NeurASP_predictions_valid_epoch_1.json") as file :
            data_JSON = json.load(file)
        file_result=file_.split("\\")[0]
        file_result+="\\"+file_.split("\\")[1]
        file_name=file_.split('\\')[-1]
        file_result3=file_result+"\\neurASP-SUP_"+file_name
        file_result
        NeurSUP(data_JSON,file_result3,sup=True,sup_Entity=True)
        print(file_result3)
        break


In [ ]:
 """

with open("Scierc/scierc_train_dev.json") as file :
    data_JSON1 = json.load(file)
    
with open("Scierc/scierc_test.json") as file :
    data_JSON2 = json.load(file)
    
    
def relations (data_JSON):
    relation_type=[]
    for document in data_JSON :
        entities_with_all=document["entities"]
        # relation(r,r1,e1,e2)
        relations_with_all=document["relations"]
        for idx ,relation_brut in enumerate(relations_with_all):
            temp= decapitalize(relation_brut["type"]).replace('-','')+","+decapitalize(entities_with_all[relation_brut["head"]]["type"])+","+decapitalize(entities_with_all[relation_brut["tail"]]["type"])
            if temp not in relation_type:
                relation_type.append(",not group(IdR,"+temp+")")
                #print("temp",temp)
    return relation_type
        
    
    
rel1 =relations (data_JSON1)
rel2 = relations (data_JSON2)
relation =list(set(rel1) | set(rel2))  
relation= ''.join(relation)
#print(relation)

def retreive_relation(relation) :
    relations= [ "Used-for", "Feature-of", "Hyponym-of", "Evaluate-for", "Part-of" , "Compare", "Conjunction","no"]
    for real in relations:
        if relation.lower()==real.replace('-','').lower():
            return real
"""

#i add relation(R,IdR,IdE1,IdE2), entity(e,IdE1,_,E1), entity(e,IdE2,_,E2), not group(IdR,no,E1,E2) from the previous test










## cleaning  ******************** 
#****************************************************************************************************************************
# les tests se font maintenant de façon automatique avec automated_evaluation.py

# we clean a files to have great NER 
import numpy as np
import glob
import json
from ast import literal_eval
def decapitalize(str):
    return str[:1].lower() + str[1:]
  

thresholds=[0.1,0.3]
with open("Scierc/scierc_test.json") as file :
    data_real = json.load(file)
types =[ "task", "method", "material", "otherScientificTerm","metric","generic"]           
#print("logs\spert_treshold_"+str(threshold)+"/*.json") 
for threshold in thresholds:
    files =glob.glob("Scierc\logs\scierc_treshold_"+str(threshold)+"\predictions*.json")
    for file_ in files :
        #file_ ="Scierc/logs/scierc_treshold_0.1/predictions_valid_epoch_1.json"
        with open(file_) as file :
            data_JSON = json.load(file)   
            print("ok")
        for doc_id, document in enumerate(data_JSON) :
            entities=document["entities"]
            for idx, entity in enumerate(entities):
                flag =True 
                for real_id, real_entity in enumerate(data_real[doc_id]["entities"])  :
                    if (entity["start"]== real_entity["start"]) and (entity["end"]== real_entity["end"]):
                        entity_probs = entity["probs"].split("[")[1].split("]")[0]
                        probs =np.fromstring(entity_probs, dtype=float, sep=' ')
                        t=len(probs)
                        #print(probs)
                        data_JSON[doc_id]["entities"][idx]["type"]=real_entity["type"]
                        probs =np.random.rand(t) * 0.1 
                        probs[1+ types.index(decapitalize(real_entity["type"]))] = 0.9
                        #print( types.index(decapitalize(real_entity["type"])))
                        data_JSON[doc_id]["entities"][idx]["probs"]=str(probs.tolist()).replace(',','')
                        flag =False 
                if flag:
                    data_JSON[doc_id]["entities"][idx]["type"]="no"
                    entity_probs = entity["probs"].split("[")[1].split("]")[0]
                    probs =np.fromstring(entity_probs, dtype=float, sep=' ')
                    probs =np.random.rand(len(probs)) * 0.01 
                    probs[0] = 0.9
                    data_JSON[doc_id]["entities"][idx]["probs"]=str(probs.tolist()).replace(',','')
        #print(data_JSON)

        with open(file_, 'w') as f:
            json.dump(data_JSON, f)
                #print("*********** You can actually test your model with evaluation functions")




                